**TENSORFLOW   + EAGER EXECUTION**

In [0]:
import os
os.environ['DISABLE_COLAB_TF_IMPORT_HOOK'] = '1'

Importing Tensorflow & Running Eager Execution

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

Import time Function to calculate Time

In [0]:
import time

MNIST data download

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("model_data/", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting model_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting model_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting model_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting model_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


MNIST data download

In [0]:
x_train = mnist.train.images
y_train =mnist.train.labels
x_test =mnist.test.images
y_test =mnist.test.labels

Reshaping the data

In [0]:
x_train = x_train.reshape(x_train.shape[0],28, 28).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28).astype('float32')

Printing shape of data

In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
y_test.shape

(55000, 28, 28)
(55000,)
(10000, 28, 28)


(10000,)

Hyper parameters

In [0]:
# Training Parameters
learning_rate = 0.001
num_steps = 20
batch_size = 128
# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit

One hot encoding

In [0]:
def scale(x, min_val=0.0, max_val=255.0):
    x = tf.to_float(x)
    return tf.div(tf.subtract(x, min_val), tf.subtract(max_val, min_val))

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(lambda x, y: (scale(x), tf.one_hot(y, 10))).shuffle(10000).batch(30)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (scale(x), tf.one_hot(y, 10))).shuffle(10000).batch(30)

Model 

In [0]:
class MNISTModel(tf.keras.Model):
    def __init__(self, device='cpu:0'):
        super(MNISTModel, self).__init__()
        self.device = device
        self._input_shape = [-1, 28, 28, 1]
        
        self.conv1 = tf.layers.Conv2D(32, 5,padding='same',activation=tf.nn.relu) # 32 filter kernal size 5
        
        self.max_pool2d = tf.layers.MaxPooling2D((2, 2), (2, 2), padding='same') # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        
        self.conv2 = tf.layers.Conv2D(64, 3,padding='same', activation=tf.nn.relu)# Convolution Layer with 64 filters and a kernel size of 3 with RELU ACTIVATION
        
        self.max_pool2d1 = tf.layers.MaxPooling2D((2, 2), (2, 2), padding='same') # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        
        # flattening layer in "call" function at this stage
        
        
        self.fc1 = tf.layers.Dense(1024)
        
        self.dropout = tf.layers.Dropout(dropout)
        
        self.fc2 = tf.layers.Dense(num_classes, activation=tf.nn.softmax)
    
    def call(self, x):
        x = tf.reshape(x, self._input_shape)
        x = self.max_pool2d(self.conv1(x))
        x = self.max_pool2d1(self.conv2(x))
        x = tf.layers.flatten(x)               #flatten here
        x = self.dropout(self.fc1(x))
        return self.fc2(x)

Loss Function

In [0]:
def loss_fn(model, x, y):
    return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(
          logits=model(x), labels=y))

In [0]:
def get_accuracy(model, x, y_true):
    logits = model(x)
    prediction = tf.argmax(logits, 1)
    equality = tf.equal(prediction, tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
    return accuracy

Need "tfe" for eager execution

In [0]:
import tensorflow.contrib.eager as tfe

Running model

In [0]:
start = time. time() #start time

In [16]:
model = MNISTModel()
optimizer = tf.train.AdamOptimizer(learning_rate)
epochs = num_steps
for (batch, (images, labels)) in enumerate(train_ds):
    with tfe.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())
    if batch % 10 == 0:
        acc = get_accuracy(model, images, labels).numpy()
        print("Iteration {}, loss: {:.3f}, train accuracy: {:.2f}%".format(batch, loss_fn(model, images, labels).numpy(), acc*100))
    if batch > epochs:
        break

Iteration 0, loss: 2.301, train accuracy: 26.67%
Iteration 10, loss: 2.300, train accuracy: 10.00%
Iteration 20, loss: 2.308, train accuracy: 6.67%


In [17]:
avg_acc = 0
test_epochs = 20
for (batch, (images, labels)) in enumerate(test_ds):
    avg_acc += get_accuracy(model, images, labels).numpy()
    if batch % 100 == 0 and batch != 0:
        print("Iteration:{}, Average test accuracy: {:.2f}%".format(batch, (avg_acc/batch)*100))
print("Final test accuracy: {:.2f}%".format(avg_acc/batch * 100))

Iteration:100, Average test accuracy: 10.93%
Iteration:200, Average test accuracy: 10.57%
Iteration:300, Average test accuracy: 10.34%
Final test accuracy: 10.12%


In [18]:
end = time. time()
a =end - start
print("The Time taken by model to run in seconds is %.2f seconds "%a)

The Time taken by model to run in seconds is 14.13 seconds 


In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            multiple                  832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  3212288   
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  10250     
Total para